In [1]:
import numpy as np
from sklearn.preprocessing import RobustScaler, OneHotEncoder
import pickle as pk
import pandas as pd
from tensorflow import keras
import scipy 
np.random.seed(0)

# Dataset With Raw X, Y, Z

In [2]:
''' 
Sliding window False
Raw X, Y, Z data 
Sampling Rate = 20 

Using 1 PID for training and 1 PID for testing
'''

with open("../data/pickles/all_pids_data_slidingwindowFalse10_samplingrate20_df.p", 'rb') as f:
  data = pk.load(f)
dataframe = data['data']
dataframe['pid'].unique()

array(['BK7610', 'BU4707', 'DC6359', 'JB3156', 'JR8022', 'MC7070',
       'SF3079'], dtype=object)

In [3]:
train_dataframe = dataframe[(dataframe['pid'] == 'SF3079') |
                            (dataframe['pid'] == 'BU4707') |
                            (dataframe['pid'] == 'DC6359') |
                            (dataframe['pid'] == 'JB3156') |
                            (dataframe['pid'] == 'JR8022') |
                            (dataframe['pid'] == 'MC7070') ]
test_dataframe = dataframe[(dataframe['pid'] == 'BK7610')]
train_dataframe.shape, test_dataframe.shape

((1679320, 6), (614460, 6))

In [4]:
train_dataframe.shape, test_dataframe.shape

((1679320, 6), (614460, 6))

In [5]:
''' 
Downsampling - basically we now take only one record per second instead of 20  
'''

def create_dataset(X, y, time_steps=1, step=1):
    Xs, ys = [], []
    for i in range(0, len(X) - time_steps, step):
        output = X.iloc[i:(i + time_steps)].values
        labels = y.iloc[i: i + time_steps]
        Xs.append(output)
        ys.append(scipy.stats.mode(labels)[0][0])
    return np.array(Xs), np.array(ys).reshape(-1, 1)


X_train, y_train = create_dataset(train_dataframe[['x', 'y', 'z']],train_dataframe.tac,20,20)
X_test, y_test = create_dataset(test_dataframe[['x', 'y', 'z']],test_dataframe.tac,20,20)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

/var/folders/7y/kyw1v_8j0g1ckfb3g6x93q1m0000gn/T/ipykernel_43436/2896753078.py:11: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  ys.append(scipy.stats.mode(labels)[0][0])
/var/folders/7y/kyw1v_8j0g1ckfb3g6x93q1m0000gn/T/ipykernel_43436/2896753078.py:11: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to Tru

((83965, 20, 3), (30722, 20, 3), (83965, 1), (30722, 1))

# LSTM on Vanilla Time Series

In [6]:
model = keras.Sequential()
model.add(keras.layers.LSTM(units=256,input_shape=[X_train.shape[1], X_train.shape[2]]))
model.add(keras.layers.Dropout(rate=0.5))
model.add(keras.layers.Dense(units=128, activation='relu'))
model.add(keras.layers.Dense(y_train.shape[1], activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['acc'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 256)               266240    
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense (Dense)               (None, 128)               32896     
                                                                 
 dense_1 (Dense)             (None, 1)                 129       
                                                                 
Total params: 299,265
Trainable params: 299,265
Non-trainable params: 0
_________________________________________________________________


In [7]:
history = model.fit(X_train, y_train,
    epochs=10,
    batch_size=32, 
    validation_split=0.0, 
    shuffle=False)   

Epoch 1/10


2023-05-11 16:16:41.309603: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


2624/2624 [==============================] - 47s 17ms/step - loss: 0.2540 - acc: 0.9407
Epoch 2/10
2624/2624 [==============================] - 47s 18ms/step - loss: 0.2732 - acc: 0.8981
Epoch 3/10
2624/2624 [==============================] - 46s 17ms/step - loss: 0.2713 - acc: 0.8985
Epoch 4/10
2624/2624 [==============================] - 46s 17ms/step - loss: 0.2849 - acc: 0.9070
Epoch 5/10
2624/2624 [==============================] - 43s 16ms/step - loss: 0.2814 - acc: 0.8932
Epoch 6/10
2624/2624 [==============================] - 44s 17ms/step - loss: 0.3170 - acc: 0.8797
Epoch 7/10
2624/2624 [==============================] - 46s 17ms/step - loss: 0.2798 - acc: 0.8896
Epoch 8/10
2624/2624 [==============================] - 43s 17ms/step - loss: 0.2116 - acc: 0.9337
Epoch 9/10
2624/2624 [==============================] - 43s 17ms/step - loss: 0.2393 - acc: 0.9110
Epoch 10/10
2624/2624 [==============================] - 43s 17ms/step - loss: 0.2766 - acc: 0.9048


In [8]:
model.evaluate(X_test, y_test) 

961/961 [==============================] - 6s 6ms/step - loss: 3.0656 - acc: 0.6347


[3.065591812133789, 0.6346917748451233]

# MLP Classifier on Vanilla Time Series

In [9]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score
clf = MLPClassifier(solver='adam', shuffle=True, random_state=1)

# X0_dash = np.reshape(X0, (X0.shape[0], X0.shape[1] * X0.shape[2]))

clf.fit(np.reshape(X_train, (83965, 60)), y_train)
# clf.get_params()

print('Accuracy ', accuracy_score(y_test, clf.predict(np.reshape(X_test, ((30722, 60))))))
print('Prec ', precision_score(y_test, clf.predict(np.reshape(X_test, ((30722, 60))))))
print('Recall ', recall_score(y_test, clf.predict(np.reshape(X_test, ((30722, 60))))))

/Users/sarah_prakriti_peters/miniconda3/envs/DL/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1098: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accuracy  0.6060803333116334
Prec  0.6285763949243873
Recall  0.927278321965229


# Bidirectional LSTM on vanilla Time Series

In [10]:
model = keras.Sequential()
model.add(keras.layers.Bidirectional(keras.layers.LSTM(units=256,input_shape=[X_train.shape[1], X_train.shape[2]])))
model.add(keras.layers.Dropout(rate=0.5))
model.add(keras.layers.Dense(units=128, activation='relu'))
model.add(keras.layers.Dense(y_train.shape[1], activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer=keras.optimizers.Adam(0.001),metrics=['acc']) 
model.build((X_train.shape)) 
model.summary() 

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional (Bidirectiona  (83965, 512)             532480    
 l)                                                              
                                                                 
 dropout_1 (Dropout)         (83965, 512)              0         
                                                                 
 dense_2 (Dense)             (83965, 128)              65664     
                                                                 
 dense_3 (Dense)             (83965, 1)                129       
                                                                 
Total params: 598,273
Trainable params: 598,273
Non-trainable params: 0
_________________________________________________________________


In [11]:
history = model.fit(
    X_train, y_train,
    epochs=25,
    batch_size=32, 
    validation_split=0.01, 
    shuffle=False)   

Epoch 1/25
2598/2598 [==============================] - 83s 31ms/step - loss: 0.3587 - acc: 0.9193 - val_loss: 0.0917 - val_acc: 0.9881
Epoch 2/25
2598/2598 [==============================] - 73s 28ms/step - loss: 0.3116 - acc: 0.8936 - val_loss: 0.1700 - val_acc: 0.9881
Epoch 3/25
2598/2598 [==============================] - 64s 25ms/step - loss: 0.2863 - acc: 0.9133 - val_loss: 0.0824 - val_acc: 0.9881
Epoch 4/25
2598/2598 [==============================] - 60s 23ms/step - loss: 0.3281 - acc: 0.8943 - val_loss: 0.1634 - val_acc: 0.9881
Epoch 5/25
2598/2598 [==============================] - 60s 23ms/step - loss: 0.3463 - acc: 0.8890 - val_loss: 0.1259 - val_acc: 0.9881
Epoch 6/25
2598/2598 [==============================] - 60s 23ms/step - loss: 0.2926 - acc: 0.9241 - val_loss: 0.1166 - val_acc: 0.9881
Epoch 7/25
2598/2598 [==============================] - 59s 23ms/step - loss: 0.3533 - acc: 0.8574 - val_loss: 0.1069 - val_acc: 0.9881
Epoch 8/25
2598/2598 [==========================

In [33]:
model.evaluate(X_test, y_test) 

351/351 [==============================] - 3s 8ms/step - loss: 0.7217 - acc: 0.7155


[0.7217283844947815, 0.7154703736305237]

# CNN On Vanilla Time Series

In [131]:
batch_size = 256
epochs = 20

flatten = keras.layers.Flatten()
conv_layer1 = keras.layers.Conv1D(filters = 64, kernel_size = 3, padding='SAME')
conv_layer2 = keras.layers.Conv1D(filters = 64, kernel_size = 3, padding='SAME')
dropout = keras.layers.Dropout(0.5)
max_pooling = keras.layers.MaxPool1D(pool_size=2)
fc_layer = keras.layers.Dense(units=128, activation = 'relu')
fc_layer2 = keras.layers.Dense(y_train.shape[1], activation = 'sigmoid')
base_model = keras.Sequential([
                                  conv_layer1,  
                                  conv_layer2, 
                                  dropout, 
                                  max_pooling, 
                                  flatten, 
                                  fc_layer, 
                                  fc_layer2
                                ])

base_model.compile(loss='binary_crossentropy', 
                   optimizer='adam',
                   metrics=['acc']) 



In [132]:
history = base_model.fit(
    X_train, y_train,
    epochs=20,
    batch_size=32, 
    validation_split=0.01,
    shuffle=False) 

Epoch 1/20
481/481 [==============================] - 3s 3ms/step - loss: 0.3220 - acc: 0.9108 - val_loss: 0.4842 - val_acc: 1.0000
Epoch 2/20
481/481 [==============================] - 2s 3ms/step - loss: 0.3256 - acc: 0.8806 - val_loss: 0.7588 - val_acc: 0.1795
Epoch 3/20
481/481 [==============================] - 2s 3ms/step - loss: 0.2737 - acc: 0.9347 - val_loss: 0.7710 - val_acc: 0.0577
Epoch 4/20
481/481 [==============================] - 1s 3ms/step - loss: 0.3076 - acc: 0.8955 - val_loss: 0.6665 - val_acc: 0.6731
Epoch 5/20
481/481 [==============================] - 2s 3ms/step - loss: 0.2746 - acc: 0.9172 - val_loss: 0.6739 - val_acc: 0.8910
Epoch 6/20
481/481 [==============================] - 1s 3ms/step - loss: 0.2521 - acc: 0.9015 - val_loss: 0.8488 - val_acc: 0.1538
Epoch 7/20
481/481 [==============================] - 1s 3ms/step - loss: 0.2694 - acc: 0.9166 - val_loss: 0.6809 - val_acc: 0.7308
Epoch 8/20
481/481 [==============================] - 1s 3ms/step - loss: 0.

In [133]:
base_model.evaluate(X_test, y_test) 

351/351 [==============================] - 0s 763us/step - loss: 0.7578 - acc: 0.3100


[0.7578141689300537, 0.3100312054157257]

# Data with Feature Engineering on Raw X, Y, Z

In [49]:
## PID = DC6359
df = pd.read_csv("../data/DC6359_feature_engg_updated_new.csv")
df = df.sample(frac=1, random_state=1).reset_index()
df = df.drop(['Unnamed: 0', 'index'], axis=1) 
df.head()

,tac,x_mean,y_mean,z_mean,x_std,y_std,z_std,x_aad,y_aad,z_aad,...,z_arg_diff,x_argmax_fft,y_argmax_fft,z_argmax_fft,x_argmin_fft,y_argmin_fft,z_argmin_fft,x_arg_diff_fft,y_arg_diff_fft,z_arg_diff_fft
0,0,-0.000585,-0.000364,0.004651,0.003299,0.004179,0.008746,0.002534,0.002625,0.005187,...,43,41,19,28,16,24,27,25,5,1
1,0,0.000710,-0.005559,-0.010026,0.043935,0.042405,0.042510,0.020880,0.019815,0.026667,...,17,1,3,1,17,42,28,16,39,27
2,1,0.023324,0.015599,-0.027149,0.128959,0.106513,0.131192,0.088268,0.074175,0.094685,...,10,3,11,6,24,16,36,21,5,30
3,0,0.012703,-0.011832,-0.009105,0.029305,0.043782,0.033665,0.020353,0.019700,0.023722,...,8,0,45,6,34,24,21,34,21,15
4,0,-0.031388,0.001648,-0.045044,0.120015,0.076248,0.114803,0.078569,0.052472,0.080129,...,5,40,0,10,24,42,9,16,42,1


In [50]:
## PID = BU4707
test_df = pd.read_csv("../data/BU4707_feature_engg_updated_new.csv")
test_df = test_df.drop(['Unnamed: 0'], axis=1) 
test_df.head()

,tac,x_mean,y_mean,z_mean,x_std,y_std,z_std,x_aad,y_aad,z_aad,...,z_arg_diff,x_argmax_fft,y_argmax_fft,z_argmax_fft,x_argmin_fft,y_argmin_fft,z_argmin_fft,x_arg_diff_fft,y_arg_diff_fft,z_arg_diff_fft
0,0,0.004578,0.007246,0.016726,0.003708,0.004323,0.004929,0.003109,0.003611,0.003963,...,22,20,23,21,24,26,31,4,3,10
1,0,0.001759,0.010015,0.016626,0.013341,0.010943,0.010745,0.006699,0.006029,0.005773,...,2,6,47,46,21,12,14,15,35,32
2,0,0.002363,0.009744,0.013846,0.017084,0.013323,0.014038,0.010165,0.008480,0.009637,...,11,6,1,4,18,24,37,12,23,33
3,0,0.002435,0.008929,0.010908,0.011838,0.009810,0.012986,0.007481,0.007362,0.009902,...,83,0,8,38,34,28,44,34,20,6
4,0,0.000526,0.009971,0.010272,0.005405,0.007611,0.010529,0.003908,0.005775,0.007740,...,5,32,16,1,35,20,26,3,4,25


In [51]:
df.shape, test_df.shape

((6216, 107), (4490, 107))

In [53]:
corr_matrix = df.corr().abs()

# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

# Find features with correlation greater than 0.95 
to_drop = [column for column in upper.columns if any(upper[column] > 0.80)]
to_drop

# Drop features 
new_df = df.drop(to_drop, axis=1)
new_test_df = test_df.drop(to_drop, axis=1)
df.shape, new_df.shape, test_df.shape, new_test_df.shape

((6216, 107), (6216, 41), (4490, 107), (4490, 41))

In [54]:
X_train = new_df.drop(['tac'], axis=1)
y_train = new_df.tac
X_test = new_test_df.drop(['tac'], axis=1)
y_test = new_test_df.tac 

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((6216, 40), (6216,), (4490, 40), (4490,))

In [57]:
y_train = np.reshape(y_train, (-1,1))
y_test = np.reshape(y_test, (-1,1))
y_train.shape, y_test.shape

((6216, 1), (4490, 1))

In [200]:
batch_size = 32
epochs = 50

fc_layer1 = keras.layers.Dense(units=128, activation = 'relu')
fc_layer2 = keras.layers.Dense(units=64, activation = 'relu')
fc_layer3 = keras.layers.Dense(units=32, activation = 'relu')
fc_layer4 = keras.layers.Dense(units=1, activation = 'sigmoid')
# Cant use softmax at the end since it will normalize and give 1 
base_model = keras.Sequential([
fc_layer1,
fc_layer2,
fc_layer3,
fc_layer4
])

base_model.compile(loss=keras.losses.BinaryCrossentropy(from_logits = False), 
                   optimizer=keras.optimizers.Adam(0.001, beta_1=0.9, beta_2= 0.999), 
                   metrics=['acc']) 

base_model.build((X_train.shape))
base_model.summary()

Model: "sequential_36"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_68 (Dense)            (83965, 20, 128)          512       
                                                                 
 dense_69 (Dense)            (83965, 20, 64)           8256      
                                                                 
 dense_70 (Dense)            (83965, 20, 32)           2080      
                                                                 
 dense_71 (Dense)            (83965, 20, 1)            33        
                                                                 
Total params: 10,881
Trainable params: 10,881
Non-trainable params: 0
_________________________________________________________________


In [94]:
# model = keras.Sequential()
# model.add(keras.layers.LSTM(20, input_shape=[40,]))
# model.add(keras.layers.TimeDistributed(keras.layers.Dense(1, activation='sigmoid')))
# model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# model.build((X_train.shape)) 
# model.summary() 

In [106]:
history = base_model.fit(
    X_train, y_train,
    epochs=20,
    batch_size=256, 
    validation_split=0.01, 
    shuffle=False)   

Epoch 1/20
25/25 [==============================] - 1s 6ms/step - loss: 0.7457 - acc: 0.7827 - val_loss: 0.7122 - val_acc: 0.7937
Epoch 2/20
25/25 [==============================] - 0s 2ms/step - loss: 0.4901 - acc: 0.8264 - val_loss: 0.5352 - val_acc: 0.7778
Epoch 3/20
25/25 [==============================] - 0s 2ms/step - loss: 0.4237 - acc: 0.8407 - val_loss: 0.5123 - val_acc: 0.8254
Epoch 4/20
25/25 [==============================] - 0s 2ms/step - loss: 0.4079 - acc: 0.8445 - val_loss: 0.4963 - val_acc: 0.8413
Epoch 5/20
25/25 [==============================] - 0s 3ms/step - loss: 0.3920 - acc: 0.8505 - val_loss: 0.5000 - val_acc: 0.8254
Epoch 6/20
25/25 [==============================] - 0s 4ms/step - loss: 0.3775 - acc: 0.8544 - val_loss: 0.4841 - val_acc: 0.8254
Epoch 7/20
25/25 [==============================] - 0s 2ms/step - loss: 0.3666 - acc: 0.8555 - val_loss: 0.4850 - val_acc: 0.8254
Epoch 8/20
25/25 [==============================] - 0s 2ms/step - loss: 0.3591 - acc: 0.85

In [107]:
base_model.evaluate(X_test, y_test)

141/141 [==============================] - 0s 527us/step - loss: 0.7980 - acc: 0.6793


[0.7980446815490723, 0.679287314414978]

# Sliding window dataset - Model does not learn on this data

In [13]:
with open("../data/pickles/data_slidingwindowTrue10_samplingrate20_df_DC6359.p", 'rb') as f:
  data = pk.load(f)
X = data['X']
Y = data['Y']
# dataframe = data['data']
# dataframe['pid'].unique() 

In [22]:
model = keras.Sequential()
model.add(
      keras.layers.LSTM(
          units=256,
          input_shape=[X.shape[1], X.shape[2]]
      )
)
model.add(keras.layers.Dropout(rate=0.5))
model.add(keras.layers.Dense(units=128, activation='relu'))
model.add(keras.layers.Dense(y_train.shape[1], activation='sigmoid'))

model.compile(
  loss='binary_crossentropy',
  optimizer='adam', 
  metrics=['acc'] 
)

model.build((X.shape))
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_5 (LSTM)               (None, 256)               266240    
                                                                 
 dropout_5 (Dropout)         (None, 256)               0         
                                                                 
 dense_10 (Dense)            (None, 128)               32896     
                                                                 
 dense_11 (Dense)            (None, 1)                 129       
                                                                 
Total params: 299,265
Trainable params: 299,265
Non-trainable params: 0
_________________________________________________________________


In [24]:
history = model.fit(
    X, Y,
    epochs=20,
    batch_size=256, 
    validation_split=0.01, 
    shuffle=False)   

Epoch 1/20
61/61 [==============================] - 38s 598ms/step - loss: 0.4650 - acc: 0.8436 - val_loss: 0.4176 - val_acc: 0.8526
Epoch 2/20
61/61 [==============================] - 38s 625ms/step - loss: 0.4206 - acc: 0.8521 - val_loss: 0.4179 - val_acc: 0.8526
Epoch 3/20
61/61 [==============================] - 37s 604ms/step - loss: 0.4185 - acc: 0.8521 - val_loss: 0.4199 - val_acc: 0.8526
Epoch 4/20
61/61 [==============================] - 37s 607ms/step - loss: 0.4181 - acc: 0.8521 - val_loss: 0.4175 - val_acc: 0.8526
Epoch 5/20
61/61 [==============================] - 47s 765ms/step - loss: 0.4183 - acc: 0.8521 - val_loss: 0.4181 - val_acc: 0.8526
Epoch 6/20
61/61 [==============================] - 37s 612ms/step - loss: 0.4196 - acc: 0.8520 - val_loss: 0.4188 - val_acc: 0.8526
Epoch 7/20
61/61 [==============================] - 37s 610ms/step - loss: 0.4202 - acc: 0.8521 - val_loss: 0.4188 - val_acc: 0.8526
Epoch 8/20
61/61 [==============================] - 38s 624ms/step - 

In [26]:
model = keras.Sequential()
model.add(keras.layers.Bidirectional(keras.layers.LSTM(units=256,input_shape=[X_train.shape[1], X_train.shape[2]])))
model.add(keras.layers.Dropout(rate=0.5))
model.add(keras.layers.Dense(units=128, activation='relu'))
model.add(keras.layers.Dense(y_train.shape[1], activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer=keras.optimizers.Adam(0.001),metrics=['acc']) 
model.build((X.shape)) 
model.summary() 

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_2 (Bidirectio  (15543, 512)             532480    
 nal)                                                            
                                                                 
 dropout_7 (Dropout)         (15543, 512)              0         
                                                                 
 dense_14 (Dense)            (15543, 128)              65664     
                                                                 
 dense_15 (Dense)            (15543, 1)                129       
                                                                 
Total params: 598,273
Trainable params: 598,273
Non-trainable params: 0
_________________________________________________________________


In [27]:
history = model.fit(
    X, Y,
    epochs=20,
    batch_size=256, 
    validation_split=0.01, 
    shuffle=False)   

Epoch 1/20
61/61 [==============================] - 48s 746ms/step - loss: 0.4486 - acc: 0.8491 - val_loss: 0.4189 - val_acc: 0.8526
Epoch 2/20
61/61 [==============================] - 46s 747ms/step - loss: 0.4176 - acc: 0.8521 - val_loss: 0.4157 - val_acc: 0.8526
Epoch 3/20
61/61 [==============================] - 46s 752ms/step - loss: 0.4174 - acc: 0.8521 - val_loss: 0.4111 - val_acc: 0.8526
Epoch 4/20
20/61 [========>.....................] - ETA: 35s - loss: 0.4428 - acc: 0.8521